# **Worksapce set up**

In [ ]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
import zipfile
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [ ]:
# Loads the data and splits it into 60% training and 40% testing sets
df = loadtxt("drive/MyDrive/GE_practicum/diabetes.csv", delimiter=",", skiprows=1)

x_train = df[0:int(0.6*len(df)),0:8]
y_train = df[0:int(0.6*len(df)),8]

x_test = df[int(0.6*len(df)):,0:8]
y_test = df[int(0.6*len(df)):,8]

# **Clustering**

In [ ]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/ANN_base')
base_model.fit(x_train,y_train,epochs = 50,validation_data = (x_test,y_test), batch_size=10)
base_model.summary()

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Epoch 1/50
46/46 [==============================] - 1s 12ms/step - loss: 0.5646 - accuracy: 0.7152 - val_loss: 0.5824 - val_accuracy: 0.7403
Epoch 2/50
46/46 [==============================] - 0s 4ms/step - loss: 0.5080 - accuracy: 0.7500 - val_loss: 0.6190 - val_accuracy: 0.7273
Epoch 3/50
46/46 [==============================] - 0s 3ms/step - loss: 0.5647 - accuracy: 0.7543 - val_loss: 0.6040 - val_accuracy: 0.7175
Epoch 4/50
46/46 [==============================] - 0s 4ms/step - loss: 0.5494 - accuracy: 0.7326 - val_loss: 0.6136 - val_accuracy: 0.7175
Epoch 5/50
46/46 [==============================] - 0s 6ms/step - loss: 0.5155 - accuracy: 0.7348 - val_loss: 0.5658 - val_accuracy: 0.7532
Epoch 6/50
46/46 [==============================] - 0s 7ms/step - loss: 0.5127 - accuracy: 0.7348 - val_loss: 0.5956 - val_accuracy: 0.7273
Epoch 7/50
46/46 [==============================] - 0s 9ms/step - loss: 0.5081 - accuracy: 0.7609 - val_loss: 0.6375 - val_accuracy: 0.7208
Epoch 8/50
46/46 [=

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

batch_size = 10
epochs = 10

num_images = x_train.shape[0]
end_step = np.ceil(num_images / batch_size).astype(np.int8) * epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

pruned_model = prune_low_magnitude(base_model, **pruning_params)

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
pruned_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])
pruned_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense_8  (None, 128)              2178      
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_9  (None, 64)               16450     
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_1  (None, 32)               4130      
 0 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_1  (None, 1)                67        
 1 (PruneLowMagnitude)                                           
                                                                 
Total params: 22,825
Trainable params: 11,521
Non-trai

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [ ]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
]

pruned_model.fit(x_train,y_train,epochs = epochs,validation_data = (x_test,y_test), batch_size=batch_size, callbacks=callbacks)
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(pruned_model, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Epoch 1/10
46/46 [==============================] - 3s 8ms/step - loss: 0.3663 - accuracy: 0.8304 - val_loss: 0.6577 - val_accuracy: 0.7435
Epoch 2/10
46/46 [==============================] - 0s 4ms/step - loss: 0.3602 - accuracy: 0.8261 - val_loss: 0.6567 - val_accuracy: 0.7435
Epoch 3/10
46/46 [==============================] - 0s 7ms/step - loss: 0.6745 - accuracy: 0.6913 - val_loss: 0.6463 - val_accuracy: 0.7078
Epoch 4/10
46/46 [==============================] - 0s 6ms/step - loss: 0.7248 - accuracy: 0.6565 - val_loss: 0.6377 - val_accuracy: 0.7078
Epoch 5/10
46/46 [==============================] - 0s 6ms/step - loss: 0.7578 - accuracy: 0.6435 - val_loss: 0.7078 - val_accuracy: 0.6753
Epoch 6/10
46/46 [==============================] - 0s 9ms/step - loss: 0.7567 - accuracy: 0.6283 - val_loss: 0.7008 - val_accuracy: 0.6721
Epoch 7/10
46/46 [==============================] - 0s 8ms/step - loss: 0.9091 - accuracy: 0.6109 - val_loss: 0.8427 - val_accuracy: 0.6981
Epoch 8/10
46/46 [==

Saved pruned Keras model to: /tmp/tmp2cj4ri6a.h5


# **Quantization aware training**

In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model
model = quantize_model(pruned_model)

opt = tf.keras.optimizers.Adam(lr = 1e-3,decay = 1e-5)
model.compile(optimizer = opt,loss = "binary_crossentropy" , metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (QuantizeL  (None, 8)                3         
 ayer)                                                           
                                                                 
 quant_dense_8 (QuantizeWrap  (None, 128)              1157      
 perV2)                                                          
                                                                 
 quant_dense_9 (QuantizeWrap  (None, 64)               8261      
 perV2)                                                          
                                                                 
 quant_dense_10 (QuantizeWra  (None, 32)               2085      
 pperV2)                                                         
                                                                 
 quant_dense_11 (QuantizeWra  (None, 1)               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Performs quantization aware training
model.fit(x_train,y_train,epochs = 50,validation_data = (x_test,y_test), batch_size=10)

Epoch 1/50
46/46 [==============================] - 3s 22ms/step - loss: 0.6634 - accuracy: 0.6196 - val_loss: 0.6012 - val_accuracy: 0.6981
Epoch 2/50
46/46 [==============================] - 0s 9ms/step - loss: 0.6261 - accuracy: 0.6217 - val_loss: 0.5718 - val_accuracy: 0.7305
Epoch 3/50
46/46 [==============================] - 0s 9ms/step - loss: 0.5752 - accuracy: 0.7109 - val_loss: 0.5094 - val_accuracy: 0.7273
Epoch 4/50
46/46 [==============================] - 0s 9ms/step - loss: 0.5594 - accuracy: 0.7043 - val_loss: 0.5023 - val_accuracy: 0.7500
Epoch 5/50
46/46 [==============================] - 0s 9ms/step - loss: 0.5355 - accuracy: 0.7196 - val_loss: 0.4980 - val_accuracy: 0.7825
Epoch 6/50
46/46 [==============================] - 0s 8ms/step - loss: 0.5375 - accuracy: 0.7304 - val_loss: 0.4645 - val_accuracy: 0.7922
Epoch 7/50
46/46 [==============================] - 0s 9ms/step - loss: 0.5318 - accuracy: 0.7283 - val_loss: 0.4738 - val_accuracy: 0.7987
Epoch 8/50
46/46 [=

# **Quantization and conversion to tflite**

In [ ]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('ANN_pruned.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmph5ctzy3k/assets


INFO:tensorflow:Assets written to: /tmp/tmph5ctzy3k/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


15736

In [ ]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#15]
  Op#0 QUANTIZE(T#0) -> [T#5]
  Op#1 FULLY_CONNECTED(T#5, T#6, T#3) -> [T#7]
  Op#2 FULLY_CONNECTED(T#7, T#8, T#4) -> [T#9]
  Op#3 FULLY_CONNECTED(T#9, T#10, T#1) -> [T#11]
  Op#4 FULLY_CONNECTED(T#11, T#12, T#2) -> [T#13]
  Op#5 LOGISTIC(T#13) -> [T#14]
  Op#6 DEQUANTIZE(T#14) -> [T#15]

Tensors of Subgraph#0
  T#0(serving_default_dense_8_input:0) shape_signature:[-1, 8], type:FLOAT32
  T#1(dense_10/bias) shape:[32], type:INT32 RO 128 bytes
  T#2(dense_11/bias) shape:[1], type:INT32 RO 4 bytes
  T#3(dense_8/bias) shape:[128], type:INT32 RO 512 bytes
  T#4(dense_9/bias) shape:[64], type:INT32 RO 256 bytes
  T#5(sequential_2/quantize_layer_1/AllValuesQuantize/FakeQuantWithMinMaxVars;quantize_layer_1/quantize_layer_1_mi

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(int(result[0]*2))

right_pred = [y_test[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.7792207792207793


# **Conversion to C array**

In [ ]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [ ]:
# Write TFLite model to a C source (or header) file
with open("ANN_pruned" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ANN_pruned"))

# **Size Comparison**

In [ ]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size('ANN_pruned.tflite')))

Size of gzipped baseline Keras model: 45047.00 bytes
Size of gzipped pruned Keras model: 16223.00 bytes
Size of gzipped pruned TFlite model: 8375.00 bytes
